# Web Scraping Tutorial
- We are using a page from amazon.com
- From that page we are extracting information like Title, Price, Ratings etc
- We are storing this information first to a dictionary then to a pandas dataframe
- Finally from the dataframe we are storing the data into a csv file


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available        

In [5]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54',
                'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [6]:
amazon_df

,title,price,rating,reviews,availability
0,Sony Playstation PS4 1TB Black Console,,4.7 out of 5 stars,897 ratings,In Stock.
1,Sony PlayStation 4 Pro 1TB Console - Black (PS...,,4.5 out of 5 stars,"4,182 ratings",Only 1 left in stock - order soon.
2,PlayStation 4 Slim 1TB Console - Marvel's Spid...,,4.7 out of 5 stars,"2,796 ratings",
3,OIVO PS4 Stand Cooling Fan Station for Playsta...,,4.5 out of 5 stars,"40,693 ratings",Not Available
4,PAC-MAN World Re-PAC - PlayStation 4,$29.99,4.8 out of 5 stars,69 ratings,In Stock.
5,"Wireless Controller for PS3, Controller for Pl...",,4.4 out of 5 stars,"1,094 ratings",Not Available
6,Marvel’s Spider-Man - Playstation 4 (PS4),$37.99,4.7 out of 5 stars,"1,859 ratings",Not Available
7,Sony Playstation 4 Slim Video Game Console 500...,,4.5 out of 5 stars,238 ratings,Not Available
8,Newest Sony Playstation 4 Slim 1TB SSD Console...,,4.5 out of 5 stars,252 ratings,Not Available
9,PlayStation 4 Slim 1TB Console,,4.7 out of 5 stars,"15,242 ratings",In Stock.


In [8]:
amazon_df.title[0:10:] # retreving first 10 entries

0               Sony Playstation PS4 1TB Black Console
1    Sony PlayStation 4 Pro 1TB Console - Black (PS...
2    PlayStation 4 Slim 1TB Console - Marvel's Spid...
3    OIVO PS4 Stand Cooling Fan Station for Playsta...
4                 PAC-MAN World Re-PAC - PlayStation 4
5    Wireless Controller for PS3, Controller for Pl...
6            Marvel’s Spider-Man - Playstation 4 (PS4)
7    Sony Playstation 4 Slim Video Game Console 500...
8    Newest Sony Playstation 4 Slim 1TB SSD Console...
9                       PlayStation 4 Slim 1TB Console
Name: title, dtype: object